In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [64]:
# Programa que permite cargar una imagen y mostrarla en pantalla con los cambios de color basados en el tipo de Daltonismo del usuario.

def cerrar_ventana(nueva_ventana):
    # Cerrar la ventana actual y mostrar la ventana principal
    nueva_ventana.destroy()
    ventana.deiconify()
def guardar_imagen(imagen_guardar):
    cv.imwrite("imagen_editada.png",imagen_guardar)
def cargar_imagen():
    global imagen_arreglo
    ruta_imagen = filedialog.askopenfilename(title="Selecciona una imagen", filetypes=[("Archivos de imagen", "*.png *.jpg *.jpeg *.gif *.bmp")])
    if ruta_imagen:
        # Abre la imagen seleccionada con Pillow (PIL)
        imagen = Image.open(ruta_imagen)
        imagen_arreglo = np.array(imagen)

        # Define el tamaño máximo en píxeles (cambiar según tus necesidades)
        ancho_maximo = 500
        alto_maximo = 300

        # Redimensiona la imagen si es necesario
        if imagen.width > ancho_maximo or imagen.height > alto_maximo:
            imagen.thumbnail((ancho_maximo, alto_maximo))

        # Convierte la imagen en un objeto Tkinter PhotoImage
        imagen_tk = ImageTk.PhotoImage(imagen)

        # Muestra la imagen en una etiqueta
        etiqueta_imagen.config(image=imagen_tk)
        etiqueta_imagen.image = imagen_tk  # ¡Importante! Mantener una referencia para evitar que se borre la imagen

        # Habilita el botón de "Mostrar Imagen"
        boton_Mostrar_Imagen.config(state="normal")
        

ventana = tk.Tk()
ventana.title("Ventana Principal")
ventana.configure(bg="#FFFFFF")
ventana.geometry("800x600")

titulo = tk.Label(ventana, text="ASISTENCIA PARA DALTONISMO", font=("Impact", 30), bg="#FFFFFF", fg="#34495E")
titulo.pack()

etiqueta_imagen = tk.Label(ventana)
etiqueta_imagen.pack()

# Crea un botón de "Cargar Imagen"
boton_cargar_imagen = tk.Button(ventana, text="Cargar Imagen", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="#34495E", command=cargar_imagen, width=60, height=10)
boton_cargar_imagen.pack()

# Crea label para introducir el tipo de daltonismo:
label_daltonismo = tk.Label(ventana, text="Tipo de daltonismo", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="#34495E")
label_daltonismo.pack()

# Crea un marco para los botones de tipo de daltonismo
marco_botones = tk.Frame(ventana, bg="#FFFFFF")
marco_botones.pack(pady=20)

# Crea los botones de tipo de daltonismo
boton_deuteranopia = tk.Button(marco_botones, text="Deuteranopia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Daltonismo(imagen_arreglo, 1), width=15, height=2)
boton_deuteranopia.pack(side="left", padx=10)
boton_deuteranomalia = tk.Button(marco_botones, text="Deuteranomalia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Daltonismo(imagen_arreglo, 2), width=15, height=2)
boton_deuteranomalia.pack(side="left", padx=10)
boton_protanopia = tk.Button(marco_botones, text="Protanopia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Daltonismo(imagen_arreglo, 3), width=15, height=2)
boton_protanopia.pack(side="left", padx=10)
boton_protanomalia = tk.Button(marco_botones, text="Protanomalia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Daltonismo(imagen_arreglo, 4), width=15, height=2)
boton_protanomalia.pack(side="left", padx=10)
boton_tritanopia = tk.Button(marco_botones, text="Tritanopia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Daltonismo(imagen_arreglo, 5), width=15, height=2)
boton_tritanopia.pack(side="left", padx=10)
boton_tritanomalia = tk.Button(marco_botones, text="Tritanomalia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Daltonismo(imagen_arreglo, 6), width=15, height=2)
boton_tritanomalia.pack(side="left", padx=10)


# Crea un marco para los botones de procesamiento de imagen
marco_botones = tk.Frame(ventana, bg="#FFFFFF")
marco_botones.pack(pady=60)


# Crea el botón de "Mostrar Imagen" inicialmente deshabilitado
boton_Mostrar_Imagen = tk.Button(marco_botones, text="Mostrar Imagen", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="#34495E", command=lambda: ImagenModificada(imagen_arreglo), state="disabled", width=15, height=2)
boton_Mostrar_Imagen.pack(side="left", padx=10)


ventana.mainloop()